## Load Environment Variables

In [27]:
import os 
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
#groq_api_key 

In [2]:
from langchain_groq import ChatGroq

llm=ChatGroq(model="llama-3.3-70b-versatile",groq_api_key=groq_api_key)
llm 

d:\chatboat conversational\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000198701B5CC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000198701B5BD0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
embeddings=HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")

In [4]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
## for blog 
'''import bs4 

loader=WebBaseLoader(
    web_paths=("https://en.wikipedia.org/wiki/Death_Note",),

    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content","post-title","post-header")
        )
    ),
)

docs=loader.load()
docs'''




'import bs4 \n\nloader=WebBaseLoader(\n    web_paths=("https://en.wikipedia.org/wiki/Death_Note",),\n\n    bs_kwargs=dict(\n        parse_only=bs4.SoupStrainer(\n            class_=("post-content","post-title","post-header")\n        )\n    ),\n)\n\ndocs=loader.load()\ndocs'

In [7]:
import bs4

loader = WebBaseLoader(
    web_paths=("https://en.wikipedia.org/wiki/Death_Note",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(id="mw-content-text")
    ),
)

docs = loader.load()

print(len(docs))
print(docs[0].page_content[:500])


1

Japanese manga series
For other uses, see Death Note (disambiguation).


Death NoteFirst tankōbon volume cover, featuring Light Yagami (front) and Ryuk (back)GenreMystery[1]Psychological thriller[2][3]Supernatural thriller[1][4]
MangaWritten byTsugumi OhbaIllustrated byTakeshi ObataPublished byShueishaEnglish publisherAUS: Madman EntertainmentNA: Viz MediaImprintJump ComicsMagazineWeekly Shōnen JumpOriginal runDecember 1, 2003 – May 15, 2006Volumes12 (List of volumes)



Further information

No


In [8]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
split=text_splitter.split_documents(docs)
vectorestore=Chroma.from_documents(documents=split,embedding=embeddings)
retriever=vectorestore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000198573150C0>, search_kwargs={})

In [10]:
system_prompt=(
    "you are an assistant for question-answering tasks. " \
    "Use the following pieces of retrieved context to answer"
    "the question.If you don't know the answer,say that you"
    "don't know. Use three sentences maximum and keep the"
    "answer concise ."
    "\n\n"
    "{context}"
)

prompt=ChatPromptTemplate.from_messages([
    ("system",system_prompt),
    ("human","{input}")
])

In [17]:
rag_chain=(
    {
        "context":retriever,
        "input":RunnablePassthrough()
    }
    |prompt
    |llm
    |StrOutputParser()
)

In [19]:
response = rag_chain.invoke(
    "Who is the main character in the story?"
)

print("\nAnswer:\n", response)



Answer:
 The main character in the story is Light. He is a complex and layered character, and his development is a key part of the plot. Light's identity as the protagonist is central to the story, which explores themes of morality, justice, and power.


## Adding chat history 

In [20]:
from langchain_core.prompts import ChatMessagePromptTemplate,MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough,RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [21]:
contextualize_q_system_prompt=(
    "Given a chat history and the latest user question, "
    "which might reference context in the chat history, "
    "formulate a standalone question that can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."

)

contextualize_q_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",contextualize_q_system_prompt),
        MessagesPlaceholder('chat_history'),
        ('human',"{input}"),
    ]
)

In [22]:
question_rewritter=(
    contextualize_q_prompt
    | llm
    |StrOutputParser()

)

In [23]:
history_aware_retriever=RunnableLambda(
    lambda x:retriever.invoke(
        question_rewritter.invoke(x)
    )
)

In [24]:
rag_chain=(
    {
        'context':history_aware_retriever,
        "input":RunnablePassthrough()
    }
    |prompt
    |llm
    |StrOutputParser()
)

In [26]:
chat_history = []

response = rag_chain.invoke(
    {
        "input": "Tell me the main Character of Death Note?",
        "chat_history": chat_history
    }
)

print(response)

# Example follow-up (history aware)
chat_history.extend([
    {"role": "human", "content": "Tell me the main Character of Death Note?"},
    {"role": "assistant", "content": response}
])

response2 = rag_chain.invoke(
    {
        "input": "Give me the summmary of story?",
        "chat_history": chat_history
    }
)

print(response2)

The main character of Death Note is Light Yagami, a brilliant yet disaffected high school student. He is the one who finds the Death Note, a mysterious black notebook with the power to kill anyone whose name is written in it. Light uses the notebook to kill high-profile criminals and create a crime-free society.
The story of Death Note follows Light Yagami, a genius high school student who discovers a mysterious notebook, the Death Note, which can kill anyone whose name is written in it. He uses the notebook to carry out a worldwide massacre of individuals he deems immoral, creating a crime-free society. The series centers on Light's actions and the efforts of an elite Japanese police task force, led by detective L, to apprehend him.
